<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Pipeline_ETL_Contenedores_Kubernetes_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🧪 Ejemplo: Pipeline ETL escalable con contenedores en AWS + Kubernetes

Este notebook documenta el diseño de un pipeline ETL distribuido, donde un contenedor ejecuta la transformación de datos almacenados en Amazon S3, y Kubernetes se encarga de escalar las instancias (pods) automáticamente según el volumen de datos.

La lógica puede adaptarse también a Google Cloud Platform (GCP) usando GCS y GKE.

---



## 📁 Código del contenedor ETL en Python

El siguiente script realiza la transformación de datos descargados desde S3 y vuelve a guardar los resultados en el bucket:

```python
# etl_transform.py

import pandas as pd
import boto3
import os

def main():
    s3 = boto3.client('s3')
    bucket = os.environ['S3_BUCKET']
    key = os.environ['S3_KEY']

    obj = s3.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(obj['Body'])

    # Transformación: convertir a mayúsculas una columna
    df['processed'] = df[df.columns[0]].apply(lambda x: str(x).upper())
    df.to_csv('/tmp/output.csv', index=False)

    s3.upload_file('/tmp/output.csv', bucket, 'output/output.csv')
    print("ETL completado.")

if __name__ == "__main__":
    main()
```



## 🐳 Dockerfile del contenedor ETL

```dockerfile
FROM python:3.9

RUN pip install pandas boto3

COPY etl_transform.py /app/
WORKDIR /app

CMD ["python", "etl_transform.py"]
```



## ☸️ YAML de despliegue en Kubernetes

El contenedor puede ejecutarse como un `Job` en Kubernetes. La configuración es:

```yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: etl-job
spec:
  template:
    spec:
      containers:
      - name: etl
        image: <aws_account_id>.dkr.ecr.<region>.amazonaws.com/etl-image:latest
        env:
        - name: S3_BUCKET
          value: "mi-bucket-etl"
        - name: S3_KEY
          value: "input/datos.csv"
      restartPolicy: Never
  backoffLimit: 4
```



## 🔄 Escalamiento automático

Para escalar el número de pods según el uso de CPU:

```bash
kubectl autoscale deployment etl-job --cpu-percent=70 --min=1 --max=10
```

Esto permite que Kubernetes despliegue más contenedores si el volumen de datos o la carga computacional lo requiere.



## 📌 Conclusión

Este pipeline demuestra cómo integrar contenedores, almacenamiento en la nube y orquestación con Kubernetes para construir soluciones escalables en Big Data. Es una arquitectura que puede implementarse tanto en AWS (EKS + S3) como en GCP (GKE + GCS).

Este enfoque promueve elasticidad, modularidad y eficiencia operacional en el procesamiento de grandes volúmenes de datos.
